In [1]:
# 1. Import necessary libraries
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
# 2. Keypoints using MP Holistic 
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
# 3. Function for mediapipe detection
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
# 4. Function for drawing landmarks
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                              mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 

In [6]:
#function for calculating the necessary angles
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
    
    return angle 

In [7]:
#function for calculating the necessary distances 
def calculate_dist(a,b):
    a = np.array(a)
    b = np.array(b)
    dist = np.linalg.norm(a - b)
    return dist

In [8]:
def n1(x):
    return ((x-0)/(180-0))

In [9]:
def n2(x):
    return ((x-0)/(33-0))

In [10]:
#function to return necessary parameters
def get_coordinates(l):
    landmarks=l
    
    # Calculate Coordinates for left parameters
    lshoulder = [landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
    lhip = [landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].y]
    lankle = [landmarks[mp_holistic.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_ANKLE.value].y]
    lknee =  [landmarks[mp_holistic.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_KNEE.value].y]
    lfootidx=[landmarks[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX.value].y]
#     lelbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
#     lwrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    
    # Calculate Coordinates for right parameters
    rshoulder = [landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
    rhip = [landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].y]
    rankle = [landmarks[mp_holistic.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_ANKLE.value].y]
    rknee =  [landmarks[mp_holistic.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_KNEE.value].y]
    rfootidx=[landmarks[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
#     relbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
#     rwrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]    
    
    # Calculate required PARAMETERS
    rhangle=calculate_angle(rshoulder,rhip,rknee)
    lhangle=calculate_angle(lshoulder,lhip,lknee)
    rkangle=calculate_angle(rankle,rknee,rhip)
    lkangle=calculate_angle(lankle,lknee,lhip)
    lfangle=calculate_angle(lknee,lankle,lfootidx)
    rfangle=calculate_angle(rknee,rankle,rfootidx)
    ldist=calculate_dist(lhip,lankle)
    rdist=calculate_dist(rhip,rankle)
#     langle = calculate_angle(lshoulder, lelbow, lwrist)
#     rangle = calculate_angle(rshoulder, relbow, rwrist)
#     lsangle=calculate_angle(lhip,lshoulder,lelbow)
#     rsangle=calculate_angle(rhip,rshoulder,relbow)
#     ankdist=calculate_dist(lankle,rankle)
#     rwdist=calculate_dist(rhip,rwrist)
#     lwdist=calculate_dist(lhip,lwrist)
    
    return rhangle,lhangle,rkangle,lkangle,lfangle,rfangle,ldist,rdist

In [17]:
def extract_keypoints(results):
    test=[]
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(141)
    if results.pose_landmarks:
        a,b,c,d,e,f,g,h = get_coordinates(results.pose_landmarks.landmark)
        test=np.array([n1(a),n1(b),n1(c),n1(d),n1(e),n1(f),g,h,n2(len(results.pose_landmarks.landmark))])
        q=np.concatenate([pose,test])
        q=q.flatten()
        return q

    else:
        return pose

In [21]:
# 4. Setup Folders for Collection

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('C:/Users/Dell/Desktop/trial') 

# Actions that we try to detect
actions = np.array(['standing', 'sitting', 'kneeling'])

# Thirty videos worth of data
no_sequences = 100

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [22]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [23]:
# 5. Collect Keypoint Values for Training and Testing

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    i=0
    # NEW LOOP
    # Loop through actions
    for action in actions:
        i+=1
        if i==1:
            k=(0,0,255)
        
        if i==2:
            k=(255,0,0)
        
        if i==3:
            k=(0,0,0)
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                
                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, k, 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, k, 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [24]:
# 6. Preprocess Data and Create Labels and Features

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [25]:
label_map = {label:num for num, label in enumerate(actions)}

In [26]:
label_map

{'standing': 0, 'sitting': 1, 'kneeling': 2}

In [27]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [28]:
np.array(sequences).shape

(300, 30, 141)

In [29]:
np.array(labels).shape

(300,)

In [30]:
X = np.array(sequences)

In [31]:
X.shape

(300, 30, 141)

In [32]:
y = to_categorical(labels).astype(int)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [34]:
y_test.shape

(15, 3)

In [35]:
# 7. Build and Train LSTM Neural Network

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

In [36]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
earlyStop=EarlyStopping(monitor="val_loss",verbose=2,mode='min',patience=3)

In [37]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,141)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [38]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [39]:
history=model.fit(X_train,y_train,epochs=2000,batch_size=10,validation_data=(X_test,y_test) ,verbose=2,callbacks=[earlyStop])

Epoch 1/2000
29/29 - 5s - loss: 0.7113 - categorical_accuracy: 0.6632 - val_loss: 0.5400 - val_categorical_accuracy: 0.8000 - 5s/epoch - 181ms/step
Epoch 2/2000
29/29 - 1s - loss: 0.6308 - categorical_accuracy: 0.8351 - val_loss: 0.5557 - val_categorical_accuracy: 0.8000 - 848ms/epoch - 29ms/step
Epoch 3/2000
29/29 - 1s - loss: 0.4631 - categorical_accuracy: 0.8596 - val_loss: 0.5124 - val_categorical_accuracy: 0.7333 - 818ms/epoch - 28ms/step
Epoch 4/2000
29/29 - 1s - loss: 0.4047 - categorical_accuracy: 0.8807 - val_loss: 0.3607 - val_categorical_accuracy: 0.8667 - 818ms/epoch - 28ms/step
Epoch 5/2000
29/29 - 1s - loss: 0.3141 - categorical_accuracy: 0.9298 - val_loss: 0.3381 - val_categorical_accuracy: 0.8000 - 871ms/epoch - 30ms/step
Epoch 6/2000
29/29 - 1s - loss: 0.6171 - categorical_accuracy: 0.9474 - val_loss: 0.3923 - val_categorical_accuracy: 0.7333 - 871ms/epoch - 30ms/step
Epoch 7/2000
29/29 - 1s - loss: 0.8708 - categorical_accuracy: 0.8632 - val_loss: 0.8650 - val_categor

In [40]:
# 9. Save Weights
model.save('action.h5')

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            52736     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 207,299
Trainable params: 207,299
Non-trai

In [42]:
# 8. Make Predictions

res = model.predict(X_test)

1/1 [==============================] - 0s 440ms/step


In [43]:
actions[np.argmax(res[4])]

'sitting'

In [44]:
actions[np.argmax(y_test[4])]

'kneeling'

In [45]:
# 10. Evaluation using Confusion Matrix and Accuracy

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [46]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 34ms/step


In [47]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [48]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[11,  0],
        [ 4,  0]],

       [[ 4,  7],
        [ 0,  4]],

       [[ 8,  0],
        [ 3,  4]]], dtype=int64)

In [49]:
accuracy_score(ytrue, yhat)

0.5333333333333333

In [50]:
print("The Accuracy is: {}% ".format(accuracy_score(ytrue, yhat)*100))

The Accuracy is: 53.333333333333336% 


In [51]:
model2 = Sequential()
model2.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,141)))
model2.add(LSTM(128, return_sequences=True, activation='relu'))
model2.add(LSTM(64, return_sequences=False, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(actions.shape[0], activation='softmax'))

In [52]:
model2.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [53]:
model2.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
9/9 [==============================] - 5s 42ms/step - loss: 0.8873 - categorical_accuracy: 0.6175
Epoch 2/2000
9/9 [==============================] - 0s 41ms/step - loss: 0.3088 - categorical_accuracy: 0.9018
Epoch 3/2000
9/9 [==============================] - 0s 40ms/step - loss: 0.0973 - categorical_accuracy: 0.9719
Epoch 4/2000
9/9 [==============================] - 0s 41ms/step - loss: 0.1035 - categorical_accuracy: 0.9649
Epoch 5/2000
9/9 [==============================] - 0s 42ms/step - loss: 0.1021 - categorical_accuracy: 0.9754
Epoch 6/2000
9/9 [==============================] - 0s 41ms/step - loss: 0.1056 - categorical_accuracy: 0.9614
Epoch 7/2000
9/9 [==============================] - 0s 41ms/step - loss: 0.2256 - categorical_accuracy: 0.9333
Epoch 8/2000
9/9 [==============================] - 0s 41ms/step - loss: 0.3457 - categorical_accuracy: 0.8526
Epoch 9/2000
9/9 [==============================] - 0s 41ms/step - loss: 0.1831 - categorical_accuracy: 0.9474
E

9/9 [==============================] - 0s 49ms/step - loss: 0.0341 - categorical_accuracy: 0.9895
Epoch 147/2000
9/9 [==============================] - 0s 48ms/step - loss: 0.0319 - categorical_accuracy: 0.9825
Epoch 148/2000
9/9 [==============================] - 0s 48ms/step - loss: 0.0548 - categorical_accuracy: 0.9789
Epoch 149/2000
9/9 [==============================] - 0s 49ms/step - loss: 0.0567 - categorical_accuracy: 0.9789
Epoch 150/2000
9/9 [==============================] - 0s 47ms/step - loss: 0.0431 - categorical_accuracy: 0.9825
Epoch 151/2000
9/9 [==============================] - 0s 48ms/step - loss: 0.0441 - categorical_accuracy: 0.9895
Epoch 152/2000
9/9 [==============================] - 0s 47ms/step - loss: 0.0323 - categorical_accuracy: 0.9895
Epoch 153/2000
9/9 [==============================] - 0s 46ms/step - loss: 0.0343 - categorical_accuracy: 0.9930
Epoch 154/2000
9/9 [==============================] - 0s 46ms/step - loss: 0.0273 - categorical_accuracy: 0.993

9/9 [==============================] - 0s 48ms/step - loss: 1.8093e-05 - categorical_accuracy: 1.0000
Epoch 289/2000
9/9 [==============================] - 0s 52ms/step - loss: 1.7640e-05 - categorical_accuracy: 1.0000
Epoch 290/2000
9/9 [==============================] - 0s 47ms/step - loss: 1.7249e-05 - categorical_accuracy: 1.0000
Epoch 291/2000
9/9 [==============================] - 0s 48ms/step - loss: 1.6799e-05 - categorical_accuracy: 1.0000
Epoch 292/2000
9/9 [==============================] - 0s 47ms/step - loss: 1.6404e-05 - categorical_accuracy: 1.0000
Epoch 293/2000
9/9 [==============================] - 0s 46ms/step - loss: 1.6045e-05 - categorical_accuracy: 1.0000
Epoch 294/2000
9/9 [==============================] - 0s 46ms/step - loss: 1.5642e-05 - categorical_accuracy: 1.0000
Epoch 295/2000
9/9 [==============================] - 0s 47ms/step - loss: 1.5354e-05 - categorical_accuracy: 1.0000
Epoch 296/2000
9/9 [==============================] - 0s 47ms/step - loss: 1.49

9/9 [==============================] - 0s 48ms/step - loss: 2.3914e-06 - categorical_accuracy: 1.0000
Epoch 429/2000
9/9 [==============================] - 0s 48ms/step - loss: 2.3714e-06 - categorical_accuracy: 1.0000
Epoch 430/2000
9/9 [==============================] - 0s 50ms/step - loss: 2.3463e-06 - categorical_accuracy: 1.0000
Epoch 431/2000
9/9 [==============================] - 0s 50ms/step - loss: 2.3283e-06 - categorical_accuracy: 1.0000
Epoch 432/2000
9/9 [==============================] - 0s 50ms/step - loss: 2.3028e-06 - categorical_accuracy: 1.0000
Epoch 433/2000
9/9 [==============================] - 0s 50ms/step - loss: 2.2773e-06 - categorical_accuracy: 1.0000
Epoch 434/2000
9/9 [==============================] - 0s 48ms/step - loss: 2.2585e-06 - categorical_accuracy: 1.0000
Epoch 435/2000
9/9 [==============================] - 0s 48ms/step - loss: 2.2371e-06 - categorical_accuracy: 1.0000
Epoch 436/2000
9/9 [==============================] - 0s 48ms/step - loss: 2.21

9/9 [==============================] - 0s 48ms/step - loss: 7.3279e-07 - categorical_accuracy: 1.0000
Epoch 569/2000
9/9 [==============================] - 0s 47ms/step - loss: 7.2693e-07 - categorical_accuracy: 1.0000
Epoch 570/2000
9/9 [==============================] - 0s 48ms/step - loss: 7.2317e-07 - categorical_accuracy: 1.0000
Epoch 571/2000
9/9 [==============================] - 0s 48ms/step - loss: 7.1647e-07 - categorical_accuracy: 1.0000
Epoch 572/2000
9/9 [==============================] - 0s 47ms/step - loss: 7.1146e-07 - categorical_accuracy: 1.0000
Epoch 573/2000
9/9 [==============================] - 0s 47ms/step - loss: 7.0727e-07 - categorical_accuracy: 1.0000
Epoch 574/2000
9/9 [==============================] - 0s 47ms/step - loss: 7.0142e-07 - categorical_accuracy: 1.0000
Epoch 575/2000
9/9 [==============================] - 0s 47ms/step - loss: 6.9891e-07 - categorical_accuracy: 1.0000
Epoch 576/2000
9/9 [==============================] - 0s 49ms/step - loss: 6.90

...

9/9 [==============================] - 0s 49ms/step - loss: 5.8559e-09 - categorical_accuracy: 1.0000
Epoch 1403/2000
9/9 [==============================] - 0s 51ms/step - loss: 5.8559e-09 - categorical_accuracy: 1.0000
Epoch 1404/2000
9/9 [==============================] - 0s 50ms/step - loss: 5.8559e-09 - categorical_accuracy: 1.0000
Epoch 1405/2000
9/9 [==============================] - 0s 51ms/step - loss: 5.8559e-09 - categorical_accuracy: 1.0000
Epoch 1406/2000
9/9 [==============================] - 0s 48ms/step - loss: 5.8559e-09 - categorical_accuracy: 1.0000
Epoch 1407/2000
9/9 [==============================] - 0s 48ms/step - loss: 5.8559e-09 - categorical_accuracy: 1.0000
Epoch 1408/2000
9/9 [==============================] - 0s 49ms/step - loss: 5.8559e-09 - categorical_accuracy: 1.0000
Epoch 1409/2000
9/9 [==============================] - 0s 49ms/step - loss: 5.8559e-09 - categorical_accuracy: 1.0000
Epoch 1410/2000
9/9 [==============================] - 0s 49ms/step - lo

9/9 [==============================] - 0s 46ms/step - loss: 2.9279e-09 - categorical_accuracy: 1.0000
Epoch 1541/2000
9/9 [==============================] - 0s 45ms/step - loss: 2.9279e-09 - categorical_accuracy: 1.0000
Epoch 1542/2000
9/9 [==============================] - 0s 46ms/step - loss: 2.5097e-09 - categorical_accuracy: 1.0000
Epoch 1543/2000
9/9 [==============================] - 0s 45ms/step - loss: 2.5097e-09 - categorical_accuracy: 1.0000
Epoch 1544/2000
9/9 [==============================] - 0s 48ms/step - loss: 2.5097e-09 - categorical_accuracy: 1.0000
Epoch 1545/2000
9/9 [==============================] - 0s 47ms/step - loss: 2.5097e-09 - categorical_accuracy: 1.0000
Epoch 1546/2000
9/9 [==============================] - 0s 48ms/step - loss: 2.5097e-09 - categorical_accuracy: 1.0000
Epoch 1547/2000
9/9 [==============================] - 0s 48ms/step - loss: 2.5097e-09 - categorical_accuracy: 1.0000
Epoch 1548/2000
9/9 [==============================] - 0s 49ms/step - lo

9/9 [==============================] - 0s 49ms/step - loss: 1.2548e-09 - categorical_accuracy: 1.0000
Epoch 1679/2000
9/9 [==============================] - 0s 50ms/step - loss: 1.2548e-09 - categorical_accuracy: 1.0000
Epoch 1680/2000
9/9 [==============================] - 0s 48ms/step - loss: 1.2548e-09 - categorical_accuracy: 1.0000
Epoch 1681/2000
9/9 [==============================] - 0s 47ms/step - loss: 1.2548e-09 - categorical_accuracy: 1.0000
Epoch 1682/2000
9/9 [==============================] - 0s 46ms/step - loss: 1.2548e-09 - categorical_accuracy: 1.0000
Epoch 1683/2000
9/9 [==============================] - 0s 47ms/step - loss: 1.2548e-09 - categorical_accuracy: 1.0000
Epoch 1684/2000
9/9 [==============================] - 0s 46ms/step - loss: 1.2548e-09 - categorical_accuracy: 1.0000
Epoch 1685/2000
9/9 [==============================] - 0s 46ms/step - loss: 1.2548e-09 - categorical_accuracy: 1.0000
Epoch 1686/2000
9/9 [==============================] - 0s 47ms/step - lo

9/9 [==============================] - 1s 58ms/step - loss: 8.3656e-10 - categorical_accuracy: 1.0000
Epoch 1817/2000
9/9 [==============================] - 0s 52ms/step - loss: 8.3656e-10 - categorical_accuracy: 1.0000
Epoch 1818/2000
9/9 [==============================] - 0s 49ms/step - loss: 8.3656e-10 - categorical_accuracy: 1.0000
Epoch 1819/2000
9/9 [==============================] - 0s 48ms/step - loss: 8.3656e-10 - categorical_accuracy: 1.0000
Epoch 1820/2000
9/9 [==============================] - 0s 48ms/step - loss: 8.3656e-10 - categorical_accuracy: 1.0000
Epoch 1821/2000
9/9 [==============================] - 0s 49ms/step - loss: 8.3656e-10 - categorical_accuracy: 1.0000
Epoch 1822/2000
9/9 [==============================] - 0s 50ms/step - loss: 8.3656e-10 - categorical_accuracy: 1.0000
Epoch 1823/2000
9/9 [==============================] - 0s 48ms/step - loss: 8.3656e-10 - categorical_accuracy: 1.0000
Epoch 1824/2000
9/9 [==============================] - 0s 55ms/step - lo

9/9 [==============================] - 0s 49ms/step - loss: 4.1828e-10 - categorical_accuracy: 1.0000
Epoch 1955/2000
9/9 [==============================] - 0s 51ms/step - loss: 4.1828e-10 - categorical_accuracy: 1.0000
Epoch 1956/2000
9/9 [==============================] - 0s 51ms/step - loss: 4.1828e-10 - categorical_accuracy: 1.0000
Epoch 1957/2000
9/9 [==============================] - 0s 50ms/step - loss: 4.1828e-10 - categorical_accuracy: 1.0000
Epoch 1958/2000
9/9 [==============================] - 0s 50ms/step - loss: 4.1828e-10 - categorical_accuracy: 1.0000
Epoch 1959/2000
9/9 [==============================] - 0s 49ms/step - loss: 4.1828e-10 - categorical_accuracy: 1.0000
Epoch 1960/2000
9/9 [==============================] - 0s 53ms/step - loss: 4.1828e-10 - categorical_accuracy: 1.0000
Epoch 1961/2000
9/9 [==============================] - 0s 49ms/step - loss: 4.1828e-10 - categorical_accuracy: 1.0000
Epoch 1962/2000
9/9 [==============================] - 0s 50ms/step - lo

In [54]:
# 9. Save Weights
model2.save('action2.h5')

In [55]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            52736     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 207,299
Trainable params: 207,299
Non-tr

In [56]:
# 8. Make Predictions

res = model2.predict(X_test)

1/1 [==============================] - 0s 475ms/step


In [57]:
actions[np.argmax(res[4])]

'kneeling'

In [58]:
actions[np.argmax(y_test[4])]

'kneeling'

In [59]:
# 10. Evaluation using Confusion Matrix and Accuracy

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [60]:
yhat = model2.predict(X_test)

1/1 [==============================] - 0s 16ms/step


In [61]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [62]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[11,  0],
        [ 0,  4]],

       [[11,  0],
        [ 0,  4]],

       [[ 8,  0],
        [ 0,  7]]], dtype=int64)

In [63]:
accuracy_score(ytrue, yhat)

1.0

In [64]:
print("The Accuracy is: {}% ".format(accuracy_score(ytrue, yhat)*100))

The Accuracy is: 100.0% 


In [68]:
# model.load_weights('action2.h5')